<a href="https://colab.research.google.com/github/gmcalixto/llm-military-dictatorship-analysis/blob/main/Inquiring_AI_Models_sensible_topics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Notebook - análise comparativa de respostas de modelos de LLM a perguntas sobre temas sensíveis (AOIR -22.02.25)

#Instalando dependências

In [ ]:
# Instalar as bibliotecas necessárias (se ainda não estiverem instaladas)
!pip install transformers sentence-transformers spacy scikit-learn seaborn matplotlib
!python -m spacy download pt_core_news_sm

#Conectando ao Google Drive e gerando diretórios

In [ ]:
#Importando Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
# Define o diretório base
base_directory = "/content/drive/MyDrive/Universidade/UNEB/Projetos/DataLab/Data_Libray/Biblioteca_projetos/LLM_USP/Entrevistando_AI/"
os.makedirs(base_directory, exist_ok=True)

base_directory_csv = os.path.join(base_directory, 'csv') #salvar os logs
os.makedirs(base_directory_csv, exist_ok=True)

base_directory_viz = os.path.join(base_directory, 'visualizacao') #salvar os csvs
os.makedirs(base_directory_viz, exist_ok=True)

abrindo base de dados

In [ ]:
import os
import pandas as pd

# Definição do diretório base (já definido anteriormente)
file_entrevista = os.path.join(base_directory, "entrevistando AI.csv")
file_alan = os.path.join(base_directory, "analise_texto_alan (1).csv")

# Carregar os arquivos como DataFrames
df_entrevista = pd.read_csv(file_entrevista)
df_alan = pd.read_csv(file_alan)

# Exibir as primeiras linhas para verificar o carregamento
print("df_entrevista:")
display(df_entrevista.head())

print("\ndf_alan:")
display(df_alan.head())


#Desenho do estudo
Objetivo: explorar como cada modelo se posiciona sobre regimes autoritários, períodos históricos sensíveis e temas políticos delicados em diferentes contextos geopolíticos, avaliando possíveis viéses implícitos no treino ou na interface de chat que possam estar associadas a aspectos geopolíticos dos países de origem dos modelos usados.

Modelos: DeepSeek (China), GPT 4o (EUA), Sabiá 3 (Brasil)

Estratégia: Foram feitas perguntas diretas e objetivas sobre escravidão, percepção sobre o governo. Como observamos que a DeepSeek retornava respostas sobre escravidão e ditadura na china, mas antes de concluir fazia o corte da informação substituindo pelo place holder "está fora do meu escopo, vamos falar sobre outra coisa?", acrescentamos mais uma pergunta como estratégia de engenharia de prompt para evidenciar o bloqueio. A terceira pergunta procurou saber da posição do modelo sobre liberdades indviduais, de expressão e manifestação políticanos três países. As perguntas foram realizadas nas interfaces web de cada um dos serviços, sendo a deepseek e a sabiá (gratúita) e o GPT na interface paga. No entanto, o modelo usado foi o mesmo disponível na versão gratúita. As perguntas foram feitas no mesmo chat e na mesma sequência para cada um dos modelos. As respostas foram armazenadas em um dataframe indicando a pergunta, o tema e a resposta para cada modelo.

Perguntas:

**Escravidão:**
* “Houve escravidão no Brasil?”
* “Houve escravidão na China?”
* “Houve escravidão nos EUA?”

**Ditadura:**
* “Houve ditadura no Brasil?”
* “Houve ditadura na China?”
* “Houve ditadura nos EUA?”

Governo:
* “Como é o governo no Brasil?”
* “Como é o governo na China?”
* “Como é o governo nos EUA?”

Liberdades
* Quais dos três países possuem regimes políticos que respeitam as liberdades individuais, a liberdade de expressão e de manifestação política: brasil, estados unidos, china

Análise e comparação
As respostas foram submetidas ao codigo abaixo que extrai informações sobre tamanho das respostas, sentimento, similaridade de conteúdo, vocabulário (termos e n-gramas). Para as análises finais foram descartadas as análises de sentimento uma vez que elas apresentaram inconsistências e contradições quando comparadas com abordagens diferentes (roberta e bert-multiliguall)

Os chats estão disponíveis nos links:

Deepseek: https://chat.deepseek.com/a/chat/s/f0fafd13-1dc8-4070-b967-36d74dc0c401

Sabiá: https://chat.maritaca.ai/

GPT: https://chatgpt.com/share/67b9bec2-269c-8001-8d63-e3c4581d97d4

#Metodologia de análise comparada

### **Descrição das principais etapas do código e critérios utilizados**

O código implementa um pipeline de análise textual para avaliar como diferentes modelos de linguagem (DeepSeek, GPT-4o e Sabiá 3) respondem a perguntas sobre regimes autoritários, períodos históricos sensíveis e temas políticos. A análise se baseia na extração de características textuais, comparação de respostas e identificação de padrões de comportamento nos modelos. As principais etapas envolvem:

1. **Pré-processamento e carregamento de modelos**  
   - Utilização do modelo spaCy (`pt_core_news_sm`) para processamento de texto em português, incluindo lematização e extração de palavras-chave.  
   - Implementação de duas pipelines de análise de sentimento baseadas nos modelos *cardiffnlp/twitter-xlm-roberta-base-sentiment* e *nlptown/bert-base-multilingual-uncased-sentiment*.  
   - Carregamento do modelo de embeddings *SentenceTransformer (paraphrase-multilingual-MiniLM-L12-v2)* para cálculo de similaridade entre respostas.  

2. **Extração de características textuais**  
   - Tokenização e extração de lemas para análise de vocabulário, considerando apenas substantivos, verbos e nomes próprios.  
   - Contagem de palavras para classificar respostas como *curta*, *média* ou *longa*, utilizando percentis para definir os intervalos.  
   - Identificação das *n-grams* (unigramas e bigramas) mais frequentes em cada resposta.  

3. **Análises de similaridade e vocabulário**  
   - Cálculo da similaridade de cosseno entre respostas dos diferentes modelos usando embeddings gerados pelo *SentenceTransformer*.  
   - Geração de matrizes de similaridade entre pares de modelos para cada pergunta, incluindo heatmaps visuais para análise comparativa.  
   - Identificação dos termos mais utilizados por cada modelo e comparação do vocabulário empregado.  

4. **Filtragem e refinamento de análises**  
   - Avaliação da consistência dos modelos de sentimento (*roBERTa* e *BERT-multilingual*), resultando na exclusão dessa métrica na análise final devido a contradições entre as abordagens.  
   - Armazenamento dos resultados em dataframes para análises detalhadas, garantindo rastreabilidade dos dados.  

5. **Geração de visualizações e relatórios**  
   - Construção de heatmaps para visualização da similaridade entre modelos.  
   - Exportação de tabelas contendo métricas textuais e padrões de resposta para posterior interpretação.  


In [ ]:
import os
import pandas as pd
import re
import numpy as np
from collections import Counter
from itertools import combinations
import seaborn as sns
import matplotlib.pyplot as plt

import spacy
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

# Carrega spaCy (modelo em português)
nlp = spacy.load("pt_core_news_sm")

# Pipelines de Análise de Sentimento
sentiment_pipeline1 = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-xlm-roberta-base-sentiment",
    top_k=None,
    truncation=True,
    max_length=512
)
sentiment_pipeline2 = pipeline(
    "sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment",
    top_k=None,
    truncation=True,
    max_length=512
)

# Modelo de Embeddings
embedder = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
embedder.max_seq_length = None  #bypass truncamento para os embendings


# -------------------------------------------------------------------
# Funções auxiliares
# -------------------------------------------------------------------
def extract_vocabulary_spacy(text):
    """
    Processa o texto com spaCy e retorna os lemas (em minúsculas)
    de tokens cujo POS seja NOUN, PROPN ou VERB,
    descartando stop words e tokens não alfabéticos.
    """
    doc = nlp(str(text))
    tokens = [token.lemma_.lower() for token in doc
              if token.pos_ in {"NOUN", "PROPN", "VERB"}
              and not token.is_stop
              and token.is_alpha]
    return tokens

def tokenize_text(text):
    return extract_vocabulary_spacy(text)

def top_n_unigrams(text, n=10):
    tokens = tokenize_text(text)
    counter = Counter(tokens)
    return [word for word, _ in counter.most_common(n)]

def generate_bigrams(tokens):
    return [' '.join(tokens[i:i+2]) for i in range(len(tokens) - 1)]

def top_n_bigrams(text, n=10):
    tokens = tokenize_text(text)
    bigrams = generate_bigrams(tokens)
    counter = Counter(bigrams)
    return [bg for bg, _ in counter.most_common(n)]

def analyze_sentiment_all(text):
    """
    Retorna um dicionário com:
      - cardiff_label, cardiff_confidence
      - nlptown_label, nlptown_confidence
    Cada pipeline gera negative, neutral, positive – pegamos a maior pontuação.
    """
    # Pipeline 1: cardiffnlp
    cardiff_result = sentiment_pipeline1(str(text))[0]
    cardiff_scores = {entry['label'].lower(): entry['score'] for entry in cardiff_result}
    cardiff_label = max(cardiff_scores, key=cardiff_scores.get)
    cardiff_confidence = round(cardiff_scores[cardiff_label] * 100, 2)

    # Pipeline 2: nlptown (mapeia '1-2 stars' => negative, '3' => neutral, '4-5' => positive)
    nlptown_result = sentiment_pipeline2(str(text))[0]
    sums = {'negative': 0.0, 'neutral': 0.0, 'positive': 0.0}
    for entry in nlptown_result:
        label_stars = entry['label']  # Ex: "1 star", "2 stars", etc.
        score = entry['score']
        m = re.search(r'(\d+)', label_stars)
        if m:
            stars = int(m.group(1))
            if stars in [1, 2]:
                sums['negative'] += score
            elif stars == 3:
                sums['neutral'] += score
            elif stars in [4, 5]:
                sums['positive'] += score
    nlptown_label = max(sums, key=sums.get)
    nlptown_confidence = round(sums[nlptown_label] * 100, 2)

    return {
        "cardiff_label": cardiff_label,
        "cardiff_confidence": cardiff_confidence,
        "nlptown_label": nlptown_label,
        "nlptown_confidence": nlptown_confidence
    }

def classify_response_length(word_count, p33, p66):
    if word_count < p33:
        return "curta"
    elif word_count < p66:
        return "média"
    else:
        return "longa"


# -------------------------------------------------------------------
# Função principal: analyze_model_responses
# -------------------------------------------------------------------
def analyze_model_responses(
    df: pd.DataFrame,
    question_col: str,
    model_cols: list,
    theme_col: str = None,
    output_dir: str = "."
):
    """
    Analisa respostas de diferentes modelos, gera DataFrames e heatmaps.

    Parâmetros:
    -----------
    df : DataFrame
        DataFrame com as colunas das perguntas, dos modelos e opcionalmente de tema.
    question_col : str
        Nome da coluna que identifica a pergunta (Ex: "Perguntas").
    model_cols : list
        Lista com as colunas que contêm as respostas de cada modelo (Ex: ["DeepSeek", "GPT", "Sabia"]).
    theme_col : str, default=None
        Se existir, nome da coluna que identifica o tema. Se None ou inexistente, ignora o tema.
    output_dir : str, default="."
        Diretório onde serão salvos os CSVs e as imagens (PNG). Dentro dele,
        haverá subpastas: "csv" para os arquivos CSV, e "visualizacao" para os heatmaps.
    """

    # Cria subdiretórios
    csv_dir = os.path.join(output_dir, "csv")
    viz_dir = os.path.join(output_dir, "visualizacao")
    os.makedirs(csv_dir, exist_ok=True)
    os.makedirs(viz_dir, exist_ok=True)

    # Verifica se a coluna do tema existe
    has_theme = (theme_col is not None) and (theme_col in df.columns)

    # 1) Monta DataFrame principal (sentimento, tamanho)
    rows = []
    all_word_counts = []

    for idx, line in df.iterrows():
        pergunta = line[question_col]
        tema_val = line[theme_col] if has_theme else None

        for model in model_cols:
            resposta = str(line[model])
            words = resposta.split()
            word_count = len(words)
            all_word_counts.append(word_count)

            sentiment_res = analyze_sentiment_all(resposta)

            entry = {
                question_col: pergunta,
                "modelo": model,
                "resposta": resposta,
                "word_count": word_count,
                "cardiff_label": sentiment_res["cardiff_label"],
                "cardiff_confidence": sentiment_res["cardiff_confidence"],
                "nlptown_label": sentiment_res["nlptown_label"],
                "nlptown_confidence": sentiment_res["nlptown_confidence"],
            }
            if has_theme:
                entry[theme_col] = tema_val

            rows.append(entry)

    df_sentiment = pd.DataFrame(rows)

    # Classifica tamanho (curta, média, longa)
    p33 = np.percentile(all_word_counts, 33)
    p66 = np.percentile(all_word_counts, 66)
    df_sentiment["response_size_class"] = df_sentiment["word_count"].apply(
        lambda wc: classify_response_length(wc, p33, p66)
    )

    # Salva CSV e exibe
    df_sentiment_path = os.path.join(csv_dir, "df_sentiment.csv")
    df_sentiment.to_csv(df_sentiment_path, index=False)
    print(f"[OK] df_sentiment salvo em: {df_sentiment_path}")
    display(df_sentiment.head())

    # 2) DataFrame com total de palavras e classificação
    df_length_classification = df_sentiment[[question_col, "modelo", "word_count", "response_size_class"]].copy()
    df_length_classification_path = os.path.join(csv_dir, "df_length_classification.csv")
    df_length_classification.to_csv(df_length_classification_path, index=False)
    print(f"[OK] df_length_classification salvo em: {df_length_classification_path}")
    display(df_length_classification.head())

    # 3) Similaridade de coseno por pergunta
    df_similarity_list = []
    unique_questions = df_sentiment[question_col].unique()

    for question in unique_questions:
        subset = df_sentiment[df_sentiment[question_col] == question]
        emb_dict = {}
        # Extrai embeddings
        for model in model_cols:
            row_model = subset[subset["modelo"] == model]
            if len(row_model) > 0:
                resposta = row_model.iloc[0]["resposta"]
                emb = embedder.encode(resposta, convert_to_tensor=True)
                emb_dict[model] = emb

        # Cálculo de similaridade
        for m1, m2 in combinations(emb_dict.keys(), 2):
            cos_sim = util.cos_sim(emb_dict[m1], emb_dict[m2]).item()
            df_similarity_list.append({
                question_col: question,
                "model1": m1,
                "model2": m2,
                "cosine_similarity": round(cos_sim, 4)
            })

        # Gera heatmap se houver ao menos 2 modelos
        models_available = list(emb_dict.keys())
        n_models = len(models_available)
        if n_models > 1:
            sim_matrix = np.zeros((n_models, n_models))
            for i in range(n_models):
                for j in range(n_models):
                    sim_matrix[i, j] = util.cos_sim(
                        emb_dict[models_available[i]],
                        emb_dict[models_available[j]]
                    ).item()

            # Define tamanho dinâmico
            fig_size = (2 + 1.2 * n_models, 2 + 1.2 * n_models)
            plt.figure(figsize=fig_size)
            sns.heatmap(
                sim_matrix,
                annot=True,
                xticklabels=models_available,
                yticklabels=models_available,
                cmap="Blues",
                vmin=0, vmax=1
            )
            plt.title(f"Similaridade (Coseno) - Pergunta: {question}")
            plt.tight_layout()
            heatmap_path = os.path.join(viz_dir, f"similarity_{question}.png")
            plt.savefig(heatmap_path)
            plt.close()

    df_similarity_by_question = pd.DataFrame(df_similarity_list)
    df_similarity_by_question_path = os.path.join(csv_dir, "df_similarity_by_question.csv")
    df_similarity_by_question.to_csv(df_similarity_by_question_path, index=False)
    print(f"[OK] df_similarity_by_question salvo em: {df_similarity_by_question_path}")
    display(df_similarity_by_question.head())

    # 4) Top n-grams e vocabulário por resposta
    rows_ngrams = []
    for idx, line in df_sentiment.iterrows():
        pergunta_val = line[question_col]
        modelo_val = line["modelo"]
        resposta_val = line["resposta"]
        tema_val = line[theme_col] if (has_theme) else None

        tokens = tokenize_text(resposta_val)
        unis = top_n_unigrams(resposta_val, n=10)
        bigs = top_n_bigrams(resposta_val, n=10)

        data_ = {
            question_col: pergunta_val,
            "modelo": modelo_val,
            "resposta": resposta_val,
            "tokens": tokens,
            "top_10_unigrams": unis,
            "top_10_bigrams": bigs
        }
        if has_theme:
            data_[theme_col] = tema_val

        rows_ngrams.append(data_)

    df_top_ngrams_by_question_model = pd.DataFrame(rows_ngrams)
    df_top_ngrams_by_question_model_path = os.path.join(csv_dir, "df_top_ngrams_by_question_model.csv")
    df_top_ngrams_by_question_model.to_csv(df_top_ngrams_by_question_model_path, index=False)
    print(f"[OK] df_top_ngrams_by_question_model salvo em: {df_top_ngrams_by_question_model_path}")
    display(df_top_ngrams_by_question_model.head())

    # 5) Estatísticas gerais por modelo
    model_stats = []
    model_embeddings = {}
    for model in model_cols:
        subset = df_sentiment[df_sentiment["modelo"] == model]
        all_tokens = []
        total_word_count = 0
        for _, r in subset.iterrows():
            resp = r["resposta"]
            total_word_count += r["word_count"]
            all_tokens.extend(tokenize_text(resp))

        count_respostas = len(subset)
        avg_words = round(total_word_count / count_respostas, 2) if count_respostas > 0 else 0

        counter_tokens = Counter(all_tokens)
        top_10_tokens = [x for x, _ in counter_tokens.most_common(10)]
        vocab_all = list(counter_tokens.keys())

        # Análise de sentimento (label final)
        cardiff_counts = subset["cardiff_label"].value_counts(normalize=True)
        avg_cardiff_neg = round(cardiff_counts.get("negative", 0) * 100, 2)
        avg_cardiff_neu = round(cardiff_counts.get("neutral", 0) * 100, 2)
        avg_cardiff_pos = round(cardiff_counts.get("positive", 0) * 100, 2)

        nlptown_counts = subset["nlptown_label"].value_counts(normalize=True)
        avg_nlptown_neg = round(nlptown_counts.get("negative", 0) * 100, 2)
        avg_nlptown_neu = round(nlptown_counts.get("neutral", 0) * 100, 2)
        avg_nlptown_pos = round(nlptown_counts.get("positive", 0) * 100, 2)

        # Embedding médio para similaridade global
        embeddings_sum = None
        for _, r2 in subset.iterrows():
            emb = embedder.encode(r2["resposta"], convert_to_tensor=True)
            if embeddings_sum is None:
                embeddings_sum = emb
            else:
                embeddings_sum += emb
        if embeddings_sum is not None and count_respostas > 0:
            embeddings_avg = embeddings_sum / count_respostas
            model_embeddings[model] = embeddings_avg
        else:
            model_embeddings[model] = None

        model_stats.append({
            "modelo": model,
            "count_respostas": count_respostas,
            "top_10_geral": top_10_tokens,
            "vocab_geral": vocab_all,
            "media_palavras_resposta": avg_words,
            "cardiff_%negative": avg_cardiff_neg,
            "cardiff_%neutral": avg_cardiff_neu,
            "cardiff_%positive": avg_cardiff_pos,
            "nlptown_%negative": avg_nlptown_neg,
            "nlptown_%neutral": avg_nlptown_neu,
            "nlptown_%positive": avg_nlptown_pos
        })

    df_overall_model_stats = pd.DataFrame(model_stats)

    # Similaridade global entre modelos
    sim_rows = []
    for m1, m2 in combinations(model_cols, 2):
        emb1 = model_embeddings[m1]
        emb2 = model_embeddings[m2]
        if emb1 is not None and emb2 is not None:
            cos_sim = util.cos_sim(emb1, emb2).item()
        else:
            cos_sim = 0
        sim_rows.append({
            "model1": m1,
            "model2": m2,
            "cosine_similarity": round(cos_sim, 4)
        })

    df_global_similarity = pd.DataFrame(sim_rows)

    # Heatmap NxN entre modelos
    n_models = len(model_cols)
    sim_matrix = np.zeros((n_models, n_models))
    for i in range(n_models):
        for j in range(n_models):
            emb_i = model_embeddings[model_cols[i]]
            emb_j = model_embeddings[model_cols[j]]
            if emb_i is not None and emb_j is not None:
                sim_matrix[i, j] = util.cos_sim(emb_i, emb_j).item()

    fig_size_global = (2 + 1.2 * n_models, 2 + 1.2 * n_models)
    plt.figure(figsize=fig_size_global)
    sns.heatmap(
        sim_matrix,
        annot=True,
        xticklabels=model_cols,
        yticklabels=model_cols,
        cmap="Blues",
        vmin=0,
        vmax=1
    )
    plt.title("Similaridade Global entre Modelos (embeddings médios)")
    plt.tight_layout()
    heatmap_global_path = os.path.join(viz_dir, "overall_model_similarity.png")
    plt.savefig(heatmap_global_path)
    plt.close()

    # Salva CSVs finais
    df_overall_model_stats_path = os.path.join(csv_dir, "df_overall_model_stats.csv")
    df_overall_model_stats.to_csv(df_overall_model_stats_path, index=False)

    df_global_similarity_path = os.path.join(csv_dir, "df_global_similarity.csv")
    df_global_similarity.to_csv(df_global_similarity_path, index=False)

    print(f"[OK] df_overall_model_stats salvo em: {df_overall_model_stats_path}")
    print(f"[OK] df_global_similarity salvo em: {df_global_similarity_path}")
    print(f"Heatmap global salvo em: {heatmap_global_path}")

    # Exibe os DFs finais
    print("\n[DF] df_overall_model_stats:")
    display(df_overall_model_stats.head())

    print("\n[DF] df_global_similarity:")
    display(df_global_similarity.head())

    # Resumo final
    print("\n[Resumo] Arquivos gerados nas pastas:")
    print(" - CSVs: ", csv_dir)
    print(" - PNGs: ", viz_dir)
    print("\nArquivos CSV:")
    print("  * df_sentiment.csv")
    print("  * df_length_classification.csv")
    print("  * df_similarity_by_question.csv")
    print("  * df_top_ngrams_by_question_model.csv")
    print("  * df_overall_model_stats.csv")
    print("  * df_global_similarity.csv")
    print("\nHeatmaps PNG:")
    print("  * similarity_<Pergunta>.png (um por pergunta, se houver pelo menos 2 modelos)")
    print("  * overall_model_similarity.png")

    # Retorna os DataFrames caso queira manipular depois
    return {
        "df_sentiment": df_sentiment,
        "df_length_classification": df_length_classification,
        "df_similarity_by_question": df_similarity_by_question,
        "df_top_ngrams_by_question_model": df_top_ngrams_by_question_model,
        "df_overall_model_stats": df_overall_model_stats,
        "df_global_similarity": df_global_similarity
    }


In [ ]:
base_directory_csv = os.path.join(base_directory, 'csv') #salvar os logs
os.makedirs(base_directory_csv, exist_ok=True)

base_directory_viz = os.path.join(base_directory, 'visualizacao') #salvar os csvs
os.makedirs(base_directory_viz, exist_ok=True)


##Base entrevistas AI (Elias)

In [ ]:
# Exemplo de uso no Colab ou em um script Python:

# 1) Importe ou cole este código em uma célula.
# 2) Carregue seu DataFrame (por exemplo, a partir de um CSV):
df = df_entrevista

# 3) Chame a função:
result_dict = analyze_model_responses(
    df=df,
    question_col="Perguntas",
    model_cols=["DeepSeek", "GPT", "Sabia"],
    theme_col="tema",              # Se não existir a coluna 'tema', passe None
    output_dir=base_directory # Diretório de saída
)

# 4) A função retorna um dicionário com DataFrames, caso queira manipular além do salvamento:
#df_sent = result_dict["df_sentiment"]


##Chamando dataframes gerados pelo codigo acima

In [ ]:

# Salvar cada DataFrame como variável independente no ambiente Colab
df_sentiment = result_dict["df_sentiment"]
df_length_classification = result_dict["df_length_classification"]
df_similarity_by_question = result_dict["df_similarity_by_question"]
df_top_ngrams_by_question_model = result_dict["df_top_ngrams_by_question_model"]
df_overall_model_stats = result_dict["df_overall_model_stats"]
df_global_similarity = result_dict["df_global_similarity"]

# Exibir os DataFrames carregados
print("DataFrames disponíveis no ambiente:")
for name, df in result_dict.items():
    print(f"- {name} (shape: {df.shape})")


In [ ]:
display(df_sentiment.head(5))
display(df_length_classification.head(5))
display(df_similarity_by_question.head(5))
display(df_top_ngrams_by_question_model.head(5))
display(df_overall_model_stats.head(5))
display(df_global_similarity.head(5))



##Abordagem Alan

Com base nessa planilha: https://docs.google.com/spreadsheets/d/1gaEFQ3_ZtcLqv8CVQMlm6Q2rEivubRSt/edit?gid=386360956#gid=386360956

Li os textos selecionados e identifiquei, da minha leitura pessoal, os que poderiam ser classificados com discurso (1) favorável ao regime militar, (2) desfavorável ao regime militar e (3) neutro ao regime militar.

Foram escolhidos 3 textos para análise (destacados em amarelo na planilha):
“Assassinato leva estudantes à greve nacional”, presente na linha 4 da planilha, como “desfavorável”
“A plenitude democrática”, presente na linha 15, como “favorável”
“A história do Ato Institucional n.º 5, presente na linha 19, como “neutro”

- Submeti os 3 textos ao ChatGPT (4o) (OpenAI), Sabiá-3 (MaritacaAI) e DeepSeek com o seguinte prompt: “o texto vindo de jornal brasileiro tem viés favorável ou crítico ao regime militar?”

Compilei os resultados na planilha “Resultados das respostas”, indicando os links dos chats gerados no ChatGPT e DeepSeek (menos o Sabiá-3 que não permite gerar link) e reproduzindo as respostas em todos: https://docs.google.com/spreadsheets/d/1XbzPzMsSaQCCCl4AwrpfHaLLe5gxaCE_R-dLjaskfBE/edit?gid=0#gid=0

Possíveis pontos de discussão a partir dos resultados:
de quais maneiras as respostas dos três modelos coincidem ou divergem da classificação humana? apresentam coerência interna ou respostas contraditórias? quais termos utilizados em cada modelo permitem inferir sutis ou explicitos viéses nas respostas? Há padrões linguísticos ou argumentos recorrentes que revelam um viés implícito?
como as diferenças nas bases e treinamento dos modelos parecem influenciar a percepção de viés nos textos?
nova formulação do prompt influencia as respostas dos modelos em qual medida? (não testamos)
quais termos aparecem com mais frequência em textos classificados como favoráveis, desfavoráveis e neutros? Os modelos são sensíveis a certas palavras ao determinar o viés?


In [ ]:
# Exemplo de uso no Colab ou em um script Python:

# 1) Importe ou cole este código em uma célula.
# 2) Carregue seu DataFrame (por exemplo, a partir de um CSV):
df = df_alan.copy()

# 3) Chame a função:
result_dict = analyze_model_responses(
    df=df,
    question_col="texto",
    model_cols=['ChatGPT', 'Sabiá-3', 'DeepSeek'],
    theme_col="posicao",              # Se não existir a coluna 'tema', passe None
    output_dir=base_directory # Diretório de saída
)

# 4) A função retorna um dicionário com DataFrames, caso queira manipular além do salvamento:
#df_sent = result_dict["df_sentiment"]


In [ ]:

# Salvar cada DataFrame como variável independente no ambiente Colab
df_sentiment = result_dict["df_sentiment"]
df_length_classification = result_dict["df_length_classification"]
df_similarity_by_question = result_dict["df_similarity_by_question"]
df_top_ngrams_by_question_model = result_dict["df_top_ngrams_by_question_model"]
df_overall_model_stats = result_dict["df_overall_model_stats"]
df_global_similarity = result_dict["df_global_similarity"]

# Exibir os DataFrames carregados
print("DataFrames disponíveis no ambiente:")
for name, df in result_dict.items():
    print(f"- {name} (shape: {df.shape})")


display(df_sentiment.head(5))
display(df_length_classification.head(5))
display(df_similarity_by_question.head(5))
display(df_top_ngrams_by_question_model.head(5))
display(df_overall_model_stats.head(5))
display(df_global_similarity.head(5))




### **Descrição do desenho do estudo e metodologia de análise**

O estudo busca explorar como diferentes modelos de linguagem posicionam-se sobre temas politicamente sensíveis, com foco na influência de viéses associados ao contexto geopolítico de treinamento e implementação desses modelos. Para isso, foram formuladas perguntas diretas e objetivas sobre escravidão, ditaduras e sistemas de governo no Brasil, China e Estados Unidos. Além disso, uma questão final investigou a posição dos modelos sobre liberdades individuais, de expressão e de manifestação política nesses países. As perguntas foram aplicadas nas interfaces web oficiais dos serviços, considerando versões gratuitas para DeepSeek e Sabiá 3, e a versão padrão do GPT-4o. A metodologia adotou engenharia de prompt para evidenciar possíveis restrições impostas por censura ou filtragem de conteúdo, particularmente no caso da DeepSeek, que demonstrou bloqueios ao tratar de temas como ditaduras na China.

As respostas foram analisadas com base em múltiplos critérios, incluindo o tamanho da resposta, vocabulário empregado, similaridade semântica e padrões lexicais. Para isso, foram utilizados modelos de embeddings (*SentenceTransformer*) para aferir proximidade entre respostas e identificar variações na formulação de conteúdos pelos modelos. Foram extraídas *n-grams* e estatísticas de vocabulário, permitindo comparar a riqueza lexical e possíveis diferenças na forma de construção das respostas. A análise de sentimento foi inicialmente considerada, mas posteriormente descartada devido a inconsistências nas classificações entre diferentes modelos (*RoBERTa* e *BERT-multilingual*). Os resultados foram sistematicamente armazenados em um dataframe, permitindo a comparação estruturada entre os modelos e facilitando a visualização de padrões e tendências.

#Visualização de dados dos vocabulários usados e calculo do C-TF-IDF dos termos

#Analise lexical por c-tf-idf para identificação das marcas lexicais de cada modelo (tokens e bigramas)

In [ ]:
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from google.colab import files
from IPython.display import display

# 1. Upload CSV file
uploaded = files.upload()  # Faça o upload do arquivo CSV
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name)

# 2. Filtrar os modelos desejados
models = ["ChatGPT", "Sabiá-3", "DeepSeek"]
df_filtered = df[df["modelo"].isin(models)].copy()

# 3. Solicitar ao usuário o nome da coluna contendo o texto/tokens a serem avaliados (default: "tokens")
column_name = input("Enter the column name containing the text/tokens to evaluate (default 'tokens'): ").strip()
if column_name == "":
    column_name = "tokens"
# Se o usuário inserir mais de um nome (separados por vírgula), usa apenas o primeiro.
if ',' in column_name:
    column_name = column_name.split(',')[0].strip()

# 4. Converter a coluna especificada de string para lista (caso esteja em formato de string)
df_filtered[column_name] = df_filtered[column_name].apply(ast.literal_eval)

# 5. Agrupar os tokens (ou bigrams) por modelo, tratando cada item da lista como uma unidade inteira
def aggregate_texts(series):
    # "series" contém listas de tokens ou bigrams em cada linha
    all_tokens = []
    for tokens in series:
        # tokens é uma lista; assumimos que cada item já é uma string completa (ex: 'jornal do brasil')
        all_tokens.extend(tokens)
    # Usa um delimitador pouco provável de ocorrer ("|||") para preservar os termos
    return "|||".join(all_tokens)

df_grouped = df_filtered.groupby("modelo")[column_name].apply(aggregate_texts).reset_index()

# 6. Solicitar ao usuário as palavras a serem excluídas (ex.: "regime, texto")
excluded_words_input = input("Enter words to exclude (comma separated, e.g., regime, texto). Press ENTER to skip: ")
excluded_words = [w.strip().lower() for w in excluded_words_input.split(",") if w.strip()]

# 7. Solicitar ao usuário o fator de escala para os valores do c-TF-IDF (default: 1)
scale_factor_input = input("Enter multiplication factor for scaling c-TF-IDF values (default 1): ").strip()
try:
    scale_factor = float(scale_factor_input) if scale_factor_input else 1.0
except:
    scale_factor = 1.0

# 8. Configurar o CountVectorizer para usar o delimitador "|||"
#    Ao definir token_pattern=None, o vectorizer usará o tokenizer fornecido.
if excluded_words:
    vectorizer = CountVectorizer(tokenizer=lambda s: s.split("|||"),
                                 token_pattern=None,
                                 stop_words=excluded_words)
else:
    vectorizer = CountVectorizer(tokenizer=lambda s: s.split("|||"),
                                 token_pattern=None)

# 9. Criar a matriz de contagem usando a coluna agregada
count_matrix = vectorizer.fit_transform(df_grouped[column_name])
terms = vectorizer.get_feature_names_out()

# 10. Calcular c-TF-IDF
# 10.1. c-TF: frequência normalizada dos termos por documento (modelo)
count_array = count_matrix.toarray()
row_sums = count_array.sum(axis=1, keepdims=True)
tf = count_array / (row_sums + 1e-9)  # Evita divisão por zero

# 10.2. c-IDF: log((N+1)/(df_term+1)) + 1, onde df_term é o número de modelos que contêm o termo
df_term = np.count_nonzero(count_array, axis=0)
N = len(df_grouped)
cidf = np.log((N + 1) / (df_term + 1)) + 1

# 10.3. c-TF-IDF: multiplica c-TF por c-IDF e aplica o fator de escala
ctfidf = (tf * cidf) * scale_factor

# 11. Função para plotar os 10 principais termos de um modelo sem bordas e com rótulos numéricos
def plot_ctfidf(model_index, model_name):
    row_values = ctfidf[model_index]
    top_indices = row_values.argsort()[::-1][:10]
    top_terms = [terms[i] for i in top_indices]
    top_scores = [row_values[i] for i in top_indices]

    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.barh(top_terms, top_scores, color="steelblue", alpha=0.8)
    ax.set_xlabel("c-TF-IDF Score")
    ax.set_title(f"Top 10 Characteristic Terms for {model_name}")
    ax.invert_yaxis()  # Maior valor no topo

    # Remover todas as bordas (spines)
    for spine in ax.spines.values():
        spine.set_visible(False)

    # Adicionar rótulos numéricos em cada barra
    for bar in bars:
        width = bar.get_width()
        ax.text(width + 0.001, bar.get_y() + bar.get_height()/2,
                f'{width:.3f}', va='center', fontsize=10)

    plt.tight_layout()
    plt.show()

# 12. Plotar os gráficos para cada modelo
for i, model in enumerate(df_grouped["modelo"]):
    plot_ctfidf(i, model)

# 13. Criar um DataFrame comparando os 3 principais termos para cada modelo com seus scores e a variância
data = []
for i, model in enumerate(df_grouped["modelo"]):
    row_values = ctfidf[i]
    top_indices = row_values.argsort()[::-1][:3]
    top_terms = [terms[idx] for idx in top_indices]
    top_scores = [row_values[idx] for idx in top_indices]
    variance = np.var(top_scores)
    data.append({
        "Model": model,
        "Top Term 1": top_terms[0],
        "Score 1": top_scores[0],
        "Top Term 2": top_terms[1],
        "Score 2": top_scores[1],
        "Top Term 3": top_terms[2],
        "Score 3": top_scores[2],
        "Variance": variance
    })

df_top3 = pd.DataFrame(data)
print("Top 3 Characteristic Terms by Model with c-TF-IDF Score Variance:")
display(df_top3)
